# AsyncIO 멀티 스크랩핑 실습

## AsyncIO 멀티 스크랩핑 실습 (1)
- 가상환경 생성
- 타겟 사이트 살펴보기
- 프로그램 설명
- async, await 설명
- 프로그램 테스트

### 환경 설정

In [ ]:
# asyncio 설치

!pip install asyncio

In [ ]:
# bs4 설치

!pip install beautifulsoup4

- [asyncio — 비동기 I/O](https://docs.python.org/ko/3/library/asyncio.html)


- `AsyncIO`는 코루틴에서 확장되어 만들어진 패키지이다.
    - `asyncio`는 `async/await` 구문을 사용하여 동시성 코드를 작성하는 라이브러리입니다.
    - 비동기 I/O을 Coroutine 으로 쉽게 작업하게 해준다.
    - `Non-blocking` 비동기 처리를 해준다.


- `Blocking I/O` : 호출된 함수가 자신의 작업을 완료될 때까지 제어권을 가지고 있다. 그래서 다른 함수는 대기하고 있어야 한다.
- `Non-blocking I/O` : 호출된 함수(서브루틴)가 `return(yield)` 후 호출한 함수(메인 루틴)에 제어권을 전달한다. 그래서 다른 함수도 일을 할 수 있다.


- [Python Requests non-blocking?](https://stackoverflow.com/questions/14245989/python-requests-non-blocking)
    - Like `urllib2`, `requests` is blocking.
    - 사용하는 함수가 blocking이면 `asyncio`를 사용하는 것보다 단일 쓰레드를 사용하는 것이 더 빠르다.


- 내가 사용하는 함수가 비동기로 구현되어야 `asyncio`를 사용할 수 있다.


- 쓰레드의 단점 : 디버깅 어려움, 지원 접근 시 레이스 컨디션(경쟁상태), 데드락(dead lock)


- 코루틴 장점 : 하나의 루틴만 실행, 락 관리 필요 없다
- 코루틴 단점 : 사용 함수가 비동기로 구현이 되어야 한다.

### Asyncio 웹 스크랩핑 실습 (1-1)

In [1]:
# aiohttp 권장
import asyncio
import timeit
from urllib.request import urlopen
from concurrent.futures import ThreadPoolExecutor
import threading

- `urlopen`은 block 함수이다.
- block 함수는 자신의 작업이 완료될 때까지 제어권을 가지고 있으므로 `asyncio`로 구현을 하면 효율이 크지 않다.


- [Combining Coroutines with Threads and Processes](https://pymotw.com/3/asyncio/executors.html)
- 블럭이 되는 함수를 사용할 때 `asyncio`를 사요앟기 위해 `corutines`을 `threads`나 `processes`와 결합해서 같이 사용하는 패턴을 많이 사용한다.
- `threads`나 `processes`를 사용하여 각 웹서버에 각각의 `threads`와 `processes`로 요청을 하면 nonblock의 효과를 가지게 된다. 
- 이 때 `courotine`으로 `asyncio`를 활용한다면 제대로된 비동기의 효과를 볼 수 있다.

### Asyncio 웹 스크랩핑 실습 (1-2)

- 데이터를 실시간으로 자동으로 수집한다
- 동시에 여러 사이트에서 수집을 한다

- 주피터 노트북에서 이벤트 루프 사용
- [Jupyter: RuntimeError: This event loop is already running](https://markhneedham.com/blog/2019/05/10/jupyter-runtimeerror-this-event-loop-is-already-running/)
- [Can't invoke asyncio event_loop after tornado 5.0 update](https://github.com/jupyter/notebook/issues/3397#issuecomment-376803076)
- The Jupyter kernel is also running an event loop, and these loops can’t be nested.


- [멀티 프로세스 모듈](https://stackoverflow.com/questions/47313732/jupyter-notebook-never-finishes-processing-using-multiprocessing-python-3)

In [6]:
!pip install nest_asyncio

In [7]:
import nest_asyncio
nest_asyncio.apply()

In [8]:
# 실행 시작 시간
start = timeit.default_timer()

# 서비스 방향이 비슷한 사이트로 실습 권장(예 : 게시판성 커뮤니티)
urls = ['http://daum.net', 'https://naver.com', 'http://mlbpark.donga.com/',
        'https://tistory.com', 'https://wemakeprice.com/']

- `urlopen` 함수는 block 함수이다.
- 쓰레드를 사용해서 `urlopen`을 따로 여러개 사용해고, `asyncio`로 제어권을 따른 곳으로 넘겨주게 만든다.

In [9]:
# fetch 함수
async def fetch(url, executor):
    # 쓰레드명 출력
    print('Thread Name :',
          threading.current_thread().getName(), 'Start', url)  # 디버깅용 코드

    # 실행
    res = await loop.run_in_executor(executor, urlopen, url)
    print('Thread Name :',
          threading.current_thread().getName(), 'Done', url)  # 디버깅용 코드

    # 결과 반환
    return res.read()[0:5]  # 각 사이트의 시작하는 5글자만 가져온다

- 제어권을 넘겨줘야 하기 때문에 `fetch` 함수를 `async`로 만들어준다.
- `url`과 `excutor(thread)`를 인자로 받는다.

```python
# 실행
res = await loop.run_in_executor(executor, urlopen, url)
```
- `await`로 끝날 때까지 기다려준다.

In [10]:
# main 함수
async def main():
    # 쓰레드 풀 생성
    executor = ThreadPoolExecutor(max_workers=10)

    # future 객체 모아서 gather에서 실행
    futures = [asyncio.ensure_future(fetch(url, executor)) for url in urls]

    # 결과 취합
    rst = await asyncio.gather(*futures)

    print()
    print('Result : ', rst)

- `async`를 붙여줌으로써 비동기 함수라고 선언을 해준다.
- `asyncio.ensure_future`에 실행할 함수들을 넣어준다.
    ```python
    # future 객체 모아서 gather에서 실행
    futures = [
        asyncio.ensure_future(fetch(url, executor)) for url in urls
    ]
    ```
    - url 하나당 하나의 쓰레드를 사용하므로 `for url in urls` 만큼 실행하게 한다.(`fetch`가 url 갯수만큼 생성된다)
- `await`는 `yield`이다.
    ```python
    # 결과 취합
    rst = await asyncio.gather(*futures)
    ```
    - `futures`가 끝날 때까지 기다린 후 결과를 취합(`gather`) 해준다.
    - 리스트이므로 `*`으로 언패킹 해준다.

In [11]:
if __name__ == '__main__':
    # 루프 초기화
    loop = asyncio.get_event_loop()  # get_event_loop()를 가져온다
    loop.run_until_complete(main())  # 작업 완료 까지 대기(끝날때까지 루프는 계속된다)

    duration = timeit.default_timer() - start  # 수행 시간 계산

    print('Total Running Time : ', duration)  # 총 실행 시간

Thread Name : MainThread Start http://daum.net
Thread Name : MainThread Start https://naver.com
Thread Name : MainThread Start http://mlbpark.donga.com/
Thread Name : MainThread Start https://tistory.com
Thread Name : MainThread Start https://wemakeprice.com/
Thread Name : MainThread Done http://mlbpark.donga.com/
Thread Name : MainThread Done https://naver.com
Thread Name : MainThread Done https://wemakeprice.com/
Thread Name : MainThread Done http://daum.net
Thread Name : MainThread Done https://tistory.com

Result :  [b'<!DOC', b'\n<!do', b'<!DOC', b'\n\t<!d', b'\x1f\x8b\x08\x00\x00']
Total Running Time :  2.280556200000092


- 실행 순서와 결과 순서가 다를 수 있다. (비동기이기 때문)
- 제어권을 넘겼기 때문에 일단 요청하고 본다.
- 먼저오는 것 순서대로 기다렸다가 `gather`로 취합을 한다.
- 그리고 `futures`에 있는 것을 언패킹하여 결과를 출력한다.
- 이를 이용하여 `fetch` 함수에 원하는 작업을 작성하면 된다.

## AsyncIO 멀티 스크랩핑 실습 (2)
- Asyncio 웹 스크랩핑 실습
- Beautiful Soup 추가
- 스케쥴러 사용시 주기적으로 데이터 수집 가능

In [12]:
import asyncio
import timeit
from urllib.request import urlopen
from concurrent.futures import ThreadPoolExecutor
import threading
from bs4 import BeautifulSoup


# 실행 시작 시간
start = timeit.default_timer()

# 서비스 방향이 비슷한 사이트로 실습 권장(예 : 게시판성 커뮤니티)
urls = ['http://daum.net', 'https://naver.com', 'http://mlbpark.donga.com/',
        'https://tistory.com', 'https://www.inflearn.com/']

In [13]:
async def fetch(url, executor):
    # 쓰레드명 출력
    print('Thread Name :', threading.current_thread().getName(), 'Start', url)
    # 실행
    res = await loop.run_in_executor(executor, urlopen, url)

    soup = BeautifulSoup(res.read(), 'html.parser')

    # 전체 페이지 소스 확인
    # print(soup.prettify())
    # 이 부분에서 BeautifulSoup Selector(선택자)를 활용해서 다양한 정보 가져오기 가능
    # 현 예제에서는 페이지 타이틀 정보 수집
    result_data = soup.title

    print('Thread Name :', threading.current_thread().getName(), 'Done', url)
    # 결과 반환
    return result_data

- BeautifulSoup를 이용하여 웹에서 가져온 데이터(`res`)를 읽어주고 `html.parser`를 이용하여 파싱한다.

In [14]:
async def main():
    # 쓰레드 풀 생성
    executor = ThreadPoolExecutor(max_workers=10)

    # future 객체 모아서 gather에서 실행
    futures = [
        asyncio.ensure_future(fetch(url, executor)) for url in urls
    ]

    # 결과 취합
    rst = await asyncio.gather(*futures)

    print()
    print('Result : ', rst)

In [15]:
if __name__ == '__main__':
    # 루프 초기화
    loop = asyncio.get_event_loop()
    # 작업 완료 까지 대기
    loop.run_until_complete(main())
    # 수행 시간 계산
    duration = timeit.default_timer() - start
    # 총 실행 시간
    print('Total Running Time : ', duration)

Thread Name : MainThread Start http://daum.net
Thread Name : MainThread Start https://naver.com
Thread Name : MainThread Start http://mlbpark.donga.com/
Thread Name : MainThread Start https://tistory.com
Thread Name : MainThread Start https://www.inflearn.com/
Thread Name : MainThread Done http://mlbpark.donga.com/
Thread Name : MainThread Done https://naver.com
Thread Name : MainThread Done https://tistory.com
Thread Name : MainThread Done https://www.inflearn.com/
Thread Name : MainThread Done http://daum.net

Result :  [<title>Daum</title>, <title>NAVER</title>, <title>↗ 파크에 오면 즐겁다 MLBPARK</title>, <title>TISTORY</title>, <title>인프런 - 1등 온라인 클래스 오픈 플랫폼</title>]
Total Running Time :  20.576115599999866
